# Data Cleaning

Dans ce notebook, nous allons nettoyer un dataset "sale" à la main avec Pandas. 

**Objectifs pédagogiques :**
1. Détecter les anomalies (Types, Doublons, Valeurs manquantes).
2. Standardiser des formats hétérogènes (Dates, Prix).
3. Corriger les erreurs de saisie (Typos, Casses).
4. Traiter les valeurs aberrantes (Outliers).

---

## 1. Chargement et Exploration Initiale

Avant de toucher aux données, il faut comprendre ce qu'on a entre les mains. 
`read_csv` est le point d'entrée classique. On utilise ensuite `info()` pour voir les types de données inférés par Pandas.

In [ ]:
import pandas as pd
import numpy as np

# Chargement du dataset
df = pd.read_csv('dirty_sales_data.csv')

# Aperçu rapide
display(df.head())

In [ ]:
df.info()

### 🔎 Analyse du problème
En regardant le résultat de `df.info()`, on remarque plusieurs red flags :
- **Date** est de type `object` (string) -> Il faudrait du `datetime`.
- **Unit_Price** est de type `object` (string) -> Il y a probablement des symboles monétaires.
- **Quantity** est bien un `int`, mais nous devrons vérifier sa cohérence métier.

## 2. Gestion des Doublons

Les doublons polluent les analyses. Il y a deux types :
1. **Doublons parfaits** : Toute la ligne est identique.
2. **Doublons partiels** : Même identifiant unique (`Transaction_ID`) mais données différentes (conflit).

Commençons par supprimer les doublons parfaits.

In [ ]:
# Compter les doublons initiaux
print(f"Doublons : {df.duplicated().sum()}")
display(df[df.duplicated(keep=False)])

In [ ]:
# Suppression des doublons exacts
df = df.drop_duplicates()
print(f"Doublons après nettoyage : {df.duplicated().sum()}")

Maintenant, vérifions si des `Transaction_ID` se répètent encore (doublons logiques).

In [ ]:
# Vérification des IDs dupliqués
duplicates_ids = df[df.duplicated(subset=['Transaction_ID'], keep=False)]
print(f"Nombre de lignes avec ID conflictuels : {len(duplicates_ids)}")

display(duplicates_ids.sort_values(by='Transaction_ID').head(10))

In [ ]:
# Stratégie : On garde la dernière occurrence (règle métier interne)
# Sinon on ferait une investigation plus poussée (timestamp, etc.)
df = df.drop_duplicates(subset=['Transaction_ID'], keep='last')
df[df.duplicated(subset=['Transaction_ID'], keep=False)]


## 3. Standardisation des Dates

La colonne `Date` contient un mélange de formats (YYYY-MM-DD vs DD/MM/YYYY) et du texte poubelle.
Utiliser simplement `pd.to_datetime(..., errors='coerce')` est risqué car cela supprimerait les dates valides au format français (ex: 18/07/2025).

**Solution Robuste :** Tenter plusieurs conversions successives.
1. D'abord le format ISO (majoritaire).
2. Ensuite le format Européen sur les échecs.
3. Ne garder en erreur que ce qui échoue aux deux.

In [ ]:
# Aperçu du problème
print("Exemples de dates avant conversion :")
print(df['Date'])

In [ ]:
# 1. Tentative Format ISO (YYYY-MM-DD)
dates_iso = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
dates_iso.head(10)

In [ ]:
# 2. Tentative Format FR (DD/MM/YYYY) sur les lignes en échec (NaT)
mask_missed = dates_iso.isna()
dates_fr = pd.to_datetime(df.loc[mask_missed, 'Date'], format='%d/%m/%Y', errors='coerce')
dates_fr.head(10)

In [ ]:
# 3. Combinaison : On remplit les trous de l'ISO avec les succès du FR
df['Date'] = dates_iso.fillna(dates_fr)
df.head(15)

In [ ]:
# Bilan
saved_docs = dates_fr.notna().sum()
final_missing = df['Date'].isnull().sum()
print(f"\nDates récupérées via format FR : {saved_docs}")
print(f"Dates irrécupérables (Vraies poubelles) : {final_missing}")

# Suppression des lignes sans date valide, indispensables pour la suite de l'analyse
df = df.dropna(subset=['Date'])
df.head(10)

## 4. Nettoyage des chaînes de caractères (Strings)

### A. Noms Clients
Les noms ont des problèmes de casse ("ACME" vs "Acme") et des espaces superflus. 
Standardisons tout en "Title Case" et retirons les espaces.

In [ ]:
df['Client_Name']

In [ ]:
df['Client_Name'].str.strip()

In [ ]:
df['Client_Name'].str.strip().str.title()

In [ ]:
# Nettoyage espaces + Title Case
df['Client_Name'] = df['Client_Name'].str.strip().str.title()

# Vérification rapide
print(df['Client_Name'].sample(5))

### B. Catégories Produits (Typos)
Regardons les valeurs uniques pour repérer les fautes de frappe.

In [ ]:
print("valeurs uniques avant :", df['Product_Category'].unique())

In [ ]:
# Dictionnaire de correction
corrections = {
    'Sofware': 'Software', 'Softwar': 'Software', 'Softwaree': 'Software',
    'Electornics': 'Electronics', 'Elec': 'Electronics', 'Electronic': 'Electronics',
    'Clothng': 'Clothing', 'Clotting': 'Clothing',
    'H0me': 'Home', 'Hom': 'Home',
    'Boks': 'Books', 'Book': 'Books'
}

# Application (replace ou map)
df['Product_Category'] = df['Product_Category'].replace(corrections)

print("\nValeurs uniques après :", df['Product_Category'].unique())

## 5. Conversions Numériques complexes (Prix)

C'est souvent le plus dur. `Unit_Price` contient des "$", "€", "USD". 
Il faut :
1. Identifier les symboles.
2. Gérer le format européen (1.200,50) vs US (1,200.50) si nécessaire.
3. Convertir en float.

*Pour simplifier ici, on va retirer tous les caractères non numériques (sauf '.' et ',') et uniformiser.*

In [ ]:
df['Unit_Price'].head()

In [ ]:
"$793 USD".replace('$', '')

In [ ]:
"$793 USD ".replace('$', '').replace('USD', '').strip()

In [ ]:
p = "1,000,000.73"
p = "1.000.00,73"
p = "10000,73"
if ',' in p and '.' in p:
    if p.rfind(',') > p.rfind('.'):
            # Format US 1,000.00 mais inversé ?? Ou juste Europe 1.000,00
            # On remplace . par rein, et , par .
            p = p.replace('.', '').replace(',', '.')
    else:
            # Format US 1,000.00 -> remove ,
            p = p.replace(',', '')
elif ',' in p:
    # Juste une virgule ? Probablement décimale si pas de point
    p = p.replace(',', '.')

p

In [ ]:
def clean_price(price_str):
    if isinstance(price_str, (int, float)):
        return float(price_str)
    
    # Convertir en string propre
    p = str(price_str)
    
    # Cas simple : on retire devises et espaces
    p = p.replace('$', '').replace('€', '').replace('USD', '').strip()
    
    # Gestion format Europe (point pour millier, virgule pour décimale)
    # Heuristique simple : si ',' est le dernier séparateur, c'est une décimale
    if ',' in p and '.' in p:
        if p.rfind(',') > p.rfind('.'):
             # Format US 1,000.00 mais inversé ?? Ou juste Europe 1.000,00
             # On remplace . par rein, et , par .
             p = p.replace('.', '').replace(',', '.')
        else:
             # Format US 1,000.00 -> remove ,
             p = p.replace(',', '')
    elif ',' in p:
        # Juste une virgule ? Probablement décimale si pas de point
        p = p.replace(',', '.')
    
    return float(p)

df['Unit_Price'] = df['Unit_Price'].apply(clean_price)
display(df.head(10))
print(df['Unit_Price'].describe())

## 6. Valeurs Aberrantes (Outliers)

La colonne `Quantity` ne doit pas être négative, et 10,000 articles d'un coup, c'est suspect.

In [ ]:
print("Avant nettoyage :")
print(df['Quantity'].describe())

In [ ]:
# Règles métier :
# 1. Quantity > 0
# 2. Quantity < 100 (seuil arbitraire)

condition_valide = (df['Quantity'] > 0) & (df['Quantity'] < 100)
condition_valide


In [ ]:
df_clean = df[condition_valide].copy()
df_clean

In [ ]:
print("\nAprès nettoyage :")
print(df_clean['Quantity'].describe())

## Conclusion

Nous avons converti un dataset "sale" en dataset propre prêt pour l'analyse.
Cependant, **c'était long et verbeux**. Il a fallu écrire une fonction spécifique pour chaque colonne.

Dans le prochain notebook, nous verrons comment un **Agent IA** peut faire tout cela pour nous en quelques secondes.

In [ ]:
df_clean.to_csv('clean_sales_data_manual.csv', index=False)
print("Fichier propre sauvegardé : clean_sales_data_manual.csv")